In [4]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import sklearn
import keras
import tensorflow as tf

print(f"Sklearn version: {sklearn.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"TensorFlow version: {tf.__version__}")

In [4]:
SR=22050
N_MFCC=40
N_MELS=128
DURATION=1
SAMPLES_PER_TRACK=SR*DURATION
#CLASSES=['AR','Sniper','nogun']

In [5]:
file_names = [
    "v3_exp1_test.csv",
    "v3_exp1_train.csv",
    "v3_exp2_test.csv",
    "v3_exp2_train.csv",
    "v3_exp3_test.csv",
    "v3_exp3_train.csv"
]

df = pd.concat([pd.read_csv(file) for file in file_names])

df.drop_duplicates(inplace=True)

In [6]:
dataset = df

In [ ]:
dataset.shape

In [9]:
def extract_features(file_path):
    y,sr=librosa.load(file_path,sr=SR,duration=DURATION)
    if len(y)<SAMPLES_PER_TRACK:
        y=np.pad(y,(0,SAMPLES_PER_TRACK-len(y)))
    else:
        y=y[:SAMPLES_PER_TRACK]
    mfcc=librosa.feature.mfcc(y=y,sr=sr,n_mfcc=N_MFCC)
    mfcc=librosa.util.fix_length(mfcc,size=174,axis=1)
    mfcc=mfcc[...,np.newaxis]

    mel_spec=librosa.feature.melspectrogram(y=y,sr=sr)
    mel_spec=librosa.power_to_db(mel_spec,ref=np.max)
    mel_spec=librosa.util.fix_length(mel_spec,size=174,axis=1)
    mel_spec=mel_spec[...,np.newaxis]
    return mfcc,mel_spec

In [10]:


def load_dataset(dataset_path, dataset):
    # Initialize lists to store features and labels
    x_mfcc, x_melspec, gun_type, direction, distance = [], [], [], [], []

    # Loop through all .wav files in the directory
    for file_name in os.listdir(dataset_path):
        if file_name.endswith(".mp3"):
            # Get the full file path
            file_path = os.path.join(dataset_path, file_name)


            # Extract features
            mfcc, mel_spec = extract_features(file_path)
            x_mfcc.append(mfcc)
            x_melspec.append(mel_spec)

            # Check if the file is in the dataset DataFrame
            row = dataset[dataset['name'] == file_name]

                # Extract gun type, direction, and distance
            gun_type.append(row['cate'].values[0])
            direction.append(row['dire'].values[0])
            distance.append(row['dist'].values[0])


    # Convert lists to numpy arrays
    x_mfcc = np.array(x_mfcc)
    x_melspec = np.array(x_melspec)

    # Encode gun type labels
    gun_encoder = LabelEncoder()
    gun_types_encoded = gun_encoder.fit_transform(gun_type)
    gun_types_categorical = to_categorical(gun_types_encoded)

    # Encode direction labels
    direction_encoder = LabelEncoder()
    direction_types_encoded = direction_encoder.fit_transform(direction)
    direction_types_categorical = to_categorical(direction_types_encoded)

    # Encode distance labels
    distance_encoder = LabelEncoder()
    distance_types_encoded = distance_encoder.fit_transform(distance)
    distance_types_categorical = to_categorical(distance_types_encoded)

    # Return the processed features and labels
    return x_mfcc, x_melspec, gun_types_categorical, direction_types_categorical, distance_types_categorical


In [11]:
X_mfcc,X_melspec,y_gun,y_direction,y_distance=load_dataset('data\gun_sound_v2',dataset)

In [ ]:
X_melspec_shape=X_melspec.shape
X_melspec_shape

In [ ]:
X_mfcc_shape=X_mfcc.shape
X_mfcc_shape

In [ ]:
print(y_gun.shape,y_direction.shape,y_distance.shape)

In [15]:
X_mfcc_train, X_mfcc_test, X_melspec_train, X_melspec_test, y_gun_train, y_gun_test,y_distance_train,y_distance_test,y_direction_train,y_direction_test = train_test_split(
    X_mfcc, X_melspec, y_gun,y_distance,y_direction,test_size=0.3, random_state=42
)

# Then, split the training+validation set into training and validation sets
# X_mfcc_train, X_mfcc_val, X_melspec_train, X_melspec_val, y_gun_train, y_gun_val,y_distance_train,y_distance_val,y_direction_train,y_direction_val = train_test_split(
#     X_mfcc_trainval, X_melspec_trainval, y_gun_trainval,y_distance_trainval,y_direction_trainval, test_size=0.3, random_state=42
# )

In [ ]:
y_gun_train=np.array(y_gun_train)
y_gun_train.shape

In [ ]:
X_mfcc_test.shape

In [18]:
def create_model(input_shape_mfcc,input_shape_melspec,num_classes=38,num_directions=6,num_distances=7):
    input_mfcc=Input(shape=input_shape_mfcc,name='mfcc_input')
    x_mfcc=Conv2D(16,kernel_size=(3,3),activation='relu')(input_mfcc)
    x_mfcc=MaxPooling2D(pool_size=(2,2))(x_mfcc)
    x_mfcc=Dropout(0.3)(x_mfcc)

    x_mfcc=Conv2D(32,kernel_size=(3,3),activation='relu')(x_mfcc)
    x_mfcc=MaxPooling2D(pool_size=(2,2))(x_mfcc)
    x_mfcc=Dropout(0.3)(x_mfcc)

    x_mfcc=Conv2D(64,kernel_size=(3,3),activation='relu')(x_mfcc)
    x_mfcc=MaxPooling2D(pool_size=(1,2))(x_mfcc)
    x_mfcc=Dropout(0.3)(x_mfcc)
    
    x_mfcc=Conv2D(128,kernel_size=(3,3),activation='relu')(x_mfcc)
    x_mfcc=MaxPooling2D(pool_size=(2,2))(x_mfcc)
    x_mfcc=Dropout(0.3)(x_mfcc)
    
    x_mfcc=GlobalAveragePooling2D()(x_mfcc)

    input_melspec=Input(shape=input_shape_melspec,name='melspec_input')
    x_melspec=Conv2D(16,kernel_size=(3,3),activation='relu')(input_melspec)
    x_melspec=MaxPooling2D(pool_size=(2,2))(x_melspec)
    x_melspec=Dropout(0.3)(x_melspec)

    x_melspec=Conv2D(32,kernel_size=(3,3),activation='relu')(x_melspec)
    x_melspec=MaxPooling2D(pool_size=(2,2))(x_melspec)
    x_melspec=Dropout(0.3)(x_melspec)

    x_melspec=Conv2D(64,kernel_size=(3,3),activation='relu')(x_melspec)
    x_melspec=MaxPooling2D(pool_size=(2,2))(x_melspec)
    x_melspec=Dropout(0.3)(x_melspec)

    x_melspec=Conv2D(128,kernel_size=(3,3),activation='relu')(x_melspec)
    x_melspec=MaxPooling2D(pool_size=(2,2))(x_melspec)
    x_melspec=Dropout(0.3)(x_melspec)

    x_melspec = GlobalAveragePooling2D()(x_melspec)

    concatenated=tf.keras.layers.concatenate([x_mfcc,x_melspec])
    common_dense=Dense(128,activation='relu',name='concatenated')(concatenated)
    gunshot_output=Dense(num_classes,activation='softmax',name='gunshot_output')(common_dense)
    direction_output=Dense(num_directions,activation='softmax',name='direction_output')(common_dense)
    distance_output=Dense(num_distances,activation='softmax',name='distance_output')(common_dense)

    model=Model(inputs=[input_mfcc,input_melspec],outputs=[gunshot_output,direction_output,distance_output])
    
    return model

In [19]:
def create_feature_extractor(input_shape_mfcc, input_shape_melspec):
    model = create_model(input_shape_mfcc, input_shape_melspec)
    feature_extractor = Model(inputs=model.inputs, outputs=model.get_layer('concatenated').output)
    return model,feature_extractor

In [20]:
input_shape_mfcc=(40,174,1)
input_shape_melspec=(128,174,1)
model,feature_extractor=create_feature_extractor(input_shape_mfcc,input_shape_melspec)

In [24]:
model.compile(optimizer='adam',
              loss={
                    'gunshot_output':'categorical_crossentropy',
                    'direction_output':'categorical_crossentropy',
                    'distance_output':'categorical_crossentropy'},
                metrics={
                    'gunshot_output': ['accuracy', AUC()],
                    'distance_output': ['accuracy', AUC()],
                    'direction_output': ['accuracy', AUC()]
                    })

In [ ]:
from keras.callbacks import ModelCheckpoint
batch_size=32
valid_batch_size=32
callbacks=[ModelCheckpoint("best_model.keras",monitor='val_loss')]

history=model.fit([X_mfcc_train,X_melspec_train],
                  {
                      'gunshot_output':y_gun_train,
                      'direction_output':y_direction_train,
                      'distance_output':y_distance_train
                  },
                  validation_data=([X_mfcc_test,X_melspec_test],
                                    {
                                        'gunshot_output':y_gun_test,
                                        'direction_output':y_direction_test,
                                        'distance_output':y_distance_test
                                    }),
                    epochs=128,batch_size=32,callbacks=[callbacks])

In [ ]:
features=feature_extractor.predict([X_mfcc,X_melspec])

In [ ]:
features

In [29]:
# layer_name='concatenated'
# intermediate_layer_model=Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
# intermediate_layer_model.summary()

In [ ]:
features.shape

In [ ]:
print(y_gun.shape,y_direction.shape,y_distance.shape)

In [32]:
y_combined = np.column_stack((y_gun, y_direction, y_distance))
X_train,X_test,y_train_combined,y_test_combined=train_test_split(features ,y_combined,test_size=0.3)

In [ ]:
y_train_combined.shape

In [34]:
y_gun_train_onehot = y_train_combined[:, :38]
y_gun_test_onehot = y_test_combined[:, :38]
y_gun_train = np.argmax(y_gun_train_onehot, axis=1)
y_gun_test = np.argmax(y_gun_test_onehot, axis=1)


In [35]:
y_dir_train_onehot = y_train_combined[:,38:44]
y_dir_test_onehot = y_test_combined[:,38:44]
y_dir_train = np.argmax(y_dir_train_onehot, axis=1)
y_dir_test = np.argmax(y_dir_test_onehot, axis=1)

In [36]:
y_dist_train_onehot = y_train_combined[:,44:51]
y_dist_test_onehot = y_test_combined[:,44:51]
y_dist_train = np.argmax(y_dist_train_onehot, axis=1)
y_dist_test = np.argmax(y_dist_test_onehot, axis=1)

In [ ]:
y_train_combined.shape

In [ ]:
X_test.shape

In [ ]:
# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Stratified K-Fold for class balance in each fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective_function_gun(params):
    model = xgb.XGBClassifier(
        n_estimators=int(params[0]),
        max_depth=int(params[1]),
        learning_rate=params[2],
        subsample=params[3],
        colsample_bytree=params[4],
        gamma=params[5],
        use_label_encoder=False,  # For newer versions of XGBoost
        eval_metric='mlogloss'    # For classification stability
    )
    # Using cross_val_score with StratifiedKFold
    cv_score = cross_val_score(model, X_train_scaled, y_gun_train, cv=skf, scoring="accuracy")
    return -cv_score.mean()  # Return negative score for minimization


In [52]:
def objective_function_dir(params):
    model = xgb.XGBClassifier(
        n_estimators=int(params[0]),
        max_depth=int(params[1]),
        learning_rate=params[2],
        subsample=params[3],
        colsample_bytree=params[4],
        gamma=params[5],
        use_label_encoder=False,  # For newer versions of XGBoost
        eval_metric='mlogloss'    # For classification stability
    )
    # Using cross_val_score with StratifiedKFold
    cv_score = cross_val_score(model, X_train_scaled, y_dir_train, cv=skf, scoring="accuracy")
    return -cv_score.mean()  # Return negative score for minimization


In [53]:
def objective_function_dist(params):
    model = xgb.XGBClassifier(
        n_estimators=int(params[0]),
        max_depth=int(params[1]),
        learning_rate=params[2],
        subsample=params[3],
        colsample_bytree=params[4],
        gamma=params[5],
        use_label_encoder=False,  # For newer versions of XGBoost
        eval_metric='mlogloss'    # For classification stability
    )
    # Using cross_val_score with StratifiedKFold
    cv_score = cross_val_score(model, X_train_scaled, y_dist_train, cv=skf, scoring="accuracy")
    return -cv_score.mean()  # Return negative score for minimization


In [54]:
param_ranges = {
    'n_estimators': (50, 500),
    'max_depth': (3, 12),
    'learning_rate': (0.01, 0.3),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'gamma': (0, 10)
}


In [55]:
import numpy as np

def initialize_population(pop_size, param_ranges):
    population = []
    for _ in range(pop_size):
        individual = [
            np.random.randint(*param_ranges['n_estimators']),
            np.random.randint(*param_ranges['max_depth']),
            np.random.uniform(*param_ranges['learning_rate']),
            np.random.uniform(*param_ranges['subsample']),
            np.random.uniform(*param_ranges['colsample_bytree']),
            np.random.uniform(*param_ranges['gamma']),
        ]
        population.append(individual)
    return np.array(population)


In [56]:
def crossover(parent1, parent2, crossover_rate=0.5):
    if np.random.rand() < crossover_rate:
        crossover_point = np.random.randint(1, len(parent1) - 1)
        child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
        child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        return child1, child2
    return parent1, parent2


In [57]:
def mutate(individual, mutation_rate=0.1):
    for i in range(len(individual)):
        if np.random.rand() < mutation_rate:  # Decide whether to mutate this parameter
            if isinstance(param_ranges[list(param_ranges.keys())[i]], tuple):  # If the range is a tuple (min, max)
                # Generate a new random float within the given range
                min_val, max_val = param_ranges[list(param_ranges.keys())[i]]
                individual[i] = np.random.uniform(min_val, max_val)
            else:  # For integer parameters
                individual[i] = np.random.randint(param_ranges[list(param_ranges.keys())[i]])
    return individual


In [58]:
def selection(population, fitness, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        # Randomly select individuals for the tournament
        tournament_indices = np.random.choice(len(population), tournament_size, replace=False)
        tournament_fitness = fitness[tournament_indices]
        # Select the best individual from the tournament
        best_index = tournament_indices[np.argmin(tournament_fitness)]
        selected.append(population[best_index])
    return np.array(selected)


In [59]:
def genetic_algorithm_gun(pop_size, generations):
    population = initialize_population(pop_size, param_ranges)
    for gen in range(generations):
        fitness = np.array([objective_function_gun(ind) for ind in population])
        selected = selection(population, fitness)
        next_gen = []
        for i in range(0, len(selected), 2):
            parent1, parent2 = selected[i], selected[i+1]
            child1, child2 = crossover(parent1, parent2)
            next_gen.append(mutate(child1))
            next_gen.append(mutate(child2))
        population = np.array(next_gen)
    best_individual = population[np.argmin([objective_function_gun(ind) for ind in population])]
    return best_individual


In [60]:
def genetic_algorithm_dir(pop_size, generations):
    population = initialize_population(pop_size, param_ranges)
    for gen in range(generations):
        fitness = np.array([objective_function_dir(ind) for ind in population])
        selected = selection(population, fitness)
        next_gen = []
        for i in range(0, len(selected), 2):
            parent1, parent2 = selected[i], selected[i+1]
            child1, child2 = crossover(parent1, parent2)
            next_gen.append(mutate(child1))
            next_gen.append(mutate(child2))
        population = np.array(next_gen)
    best_individual = population[np.argmin([objective_function_dir(ind) for ind in population])]
    return best_individual


In [61]:
def genetic_algorithm_dist(pop_size, generations):
    population = initialize_population(pop_size, param_ranges)
    for gen in range(generations):
        fitness = np.array([objective_function_dist(ind) for ind in population])
        selected = selection(population, fitness)
        next_gen = []
        for i in range(0, len(selected), 2):
            parent1, parent2 = selected[i], selected[i+1]
            child1, child2 = crossover(parent1, parent2)
            next_gen.append(mutate(child1))
            next_gen.append(mutate(child2))
        population = np.array(next_gen)
    best_individual = population[np.argmin([objective_function_dist(ind) for ind in population])]
    return best_individual


In [ ]:
ans_gun=genetic_algorithm_gun(6,10)

In [ ]:
print("Optimized Hyperparameters:")
print(f"n_estimators: {ans_gun[0]}")
print(f"max_depth: {ans_gun[1]}")
print(f"learning_rate: {ans_gun[2]}")
print(f"subsample: {ans_gun[3]}")
print(f"colsample_bytree: {ans_gun[4]}")
print(f"gamma: {ans_gun[5]}")

In [ ]:
ans_dir=genetic_algorithm_dir(8,10)
print("Optimized Hyperparameters:")
print(f"n_estimators: {ans_dir[0]}")
print(f"max_depth: {ans_dir[1]}")
print(f"learning_rate: {ans_dir[2]}")
print(f"subsample: {ans_dir[3]}")
print(f"colsample_bytree: {ans_dir[4]}")
print(f"gamma: {ans_dir[5]}")

In [ ]:
ans_dist=genetic_algorithm_dist(6,10)
print("Optimized Hyperparameters:")
print(f"n_estimators: {ans_dist[0]}")
print(f"max_depth: {ans_dist[1]}")
print(f"learning_rate: {ans_dist[2]}")
print(f"subsample: {ans_dist[3]}")
print(f"colsample_bytree: {ans_dist[4]}")
print(f"gamma: {ans_dist[5]}")

In [ ]:
import xgboost as xgb
xgb_model_gun=xgb.XGBClassifier(
        n_estimators=int(ans_gun[0]),
        max_depth=int(ans_gun[1]),
        learning_rate=ans_gun[2],
        subsample=ans_gun[3],
        colsample_bytree=ans_gun[4],
        gamma=ans_gun[5],
        use_label_encoder=False,  
        eval_metric='mlogloss'   
    )
xgb_model_gun.fit(X_train,y_gun_train)

In [ ]:

xgb_model_direction = xgb.XGBClassifier(
        n_estimators=int(ans_dir[0]),
        max_depth=int(ans_dir[1]),
        learning_rate=ans_dir[2],
        subsample=ans_dir[3],
        colsample_bytree=ans_dir[4],
        gamma=ans_dir[5],
        use_label_encoder=False,  
        eval_metric='mlogloss'    
    )
xgb_model_direction.fit(X_train, y_dir_train)

In [ ]:

xgb_model_distance = xgb.XGBClassifier(
        n_estimators=int(ans_dist[0]),
        max_depth=int(ans_dist[1]),
        learning_rate=ans_dist[2],
        subsample=ans_dist[3],
        colsample_bytree=ans_dist[4],
        gamma=ans_dist[5],
        use_label_encoder=False,  
        eval_metric='mlogloss',   
    )
xgb_model_distance.fit(X_train, y_dist_train)

In [339]:
gun_type_pred=xgb_model_gun.predict(X_test)

In [340]:
direction_type_pred = xgb_model_direction.predict(X_test)

In [341]:

distance_type_pred = xgb_model_distance.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

# Evaluate accuracy for gun type
gun_accuracy = accuracy_score(y_gun_test, gun_type_pred)
print(f"Accuracy for gun type classification: {gun_accuracy * 100:.2f}%")

In [ ]:
from sklearn.metrics import accuracy_score
direction_accuracy = accuracy_score(y_dir_test, direction_type_pred)
print(f"Accuracy for direction type classification: {direction_accuracy * 100:.2f}%")

In [ ]:
distance_accuracy = accuracy_score(y_dist_test, distance_type_pred)
print(f"Accuracy for distance type classification: {distance_accuracy * 100:.2f}%")

In [ ]:
from sklearn.metrics import classification_report

# Gun type classification report
print("Gun Type Classification Report")
print(classification_report(y_gun_test, gun_type_pred))

# Direction type classification report
print("Direction Type Classification Report")
print(classification_report(y_dir_test, direction_type_pred))

# Distance type classification report
print("Distance Type Classification Report")
print(classification_report(y_dist_test, distance_type_pred))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm_gun = confusion_matrix(y_gun_test, gun_type_pred)
sns.heatmap(cm_gun, annot=True, fmt='d', cmap='Blues')
plt.title("Gun Type Confusion Matrix")
plt.show()

In [ ]:
cm_direction = confusion_matrix(y_dir_test, direction_type_pred)
sns.heatmap(cm_direction, annot=True, fmt='d', cmap='Blues')
plt.title("Direction Type Confusion Matrix")
plt.show()

In [ ]:
cm_distance = confusion_matrix(y_dist_test, distance_type_pred)
sns.heatmap(cm_distance, annot=True, fmt='d', cmap='Blues')
plt.title("Distance Type Confusion Matrix")
plt.show()

In [ ]:
import plotly.graph_objects as go
plt.clf()
fig = go.Figure()
fig.add_trace(go.Scatter(
                    y=history.history['gunshot_output_accuracy'],
                    name='Train'))
fig.add_trace(go.Scatter(
                    y=history.history['val_gunshot_output_accuracy'],
                    name='Valid'))
fig.update_layout(height=500, 
                  width=700,
                  title='Accuracy for gunshot feature',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy')
fig.show()

In [ ]:
plt.clf()
fig = go.Figure()
fig.add_trace(go.Scatter(
                    y=history.history['direction_output_accuracy'],
                    name='Train'))
fig.add_trace(go.Scatter(
                    y=history.history['val_direction_output_accuracy'],
                    name='Valid'))
fig.update_layout(height=500, 
                  width=700,
                  title='Accuracy for direction feature',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy')
fig.show()

In [ ]:
plt.clf()
fig = go.Figure()
fig.add_trace(go.Scatter(
                    y=history.history['distance_output_accuracy'],
                    name='Train'))
fig.add_trace(go.Scatter(
                    y=history.history['val_distance_output_accuracy'],
                    name='Valid'))
fig.update_layout(height=500, 
                  width=700,
                  title='Accuracy for distance feature',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy')
fig.show()

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history, model_name):
    metrics = ["accuracy"]
    for metric in metrics:

        plt.plot(
            history.history[f"val_{model_name}_{metric}"],
            label=f"Validation {metric}",
        )
        plt.title(f"{model_name.capitalize()} {metric.capitalize()} Over Epochs")
        plt.xlabel("Epochs")
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.grid()
        plt.show()

# Plot training history for each output of the neural network
plot_training_history(history, "gunshot_output")
plot_training_history(history, "direction_output")
plot_training_history(history, "distance_output")


In [314]:
history_df = pd.DataFrame(history.history)
history_df.to_csv('model1.csv', index=False)